In [1]:
import numpy as np
import pandas as pd
import pickle as p
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import os

In [2]:
dir = "../results3"

In [3]:
files = os.listdir(dir)

In [4]:
architecture = np.array([128, 10])

In [5]:
def avg_usage(rates, architecture):
    weights = architecture / np.sum(architecture)
    avg = np.dot(rates, weights)
    return avg

In [6]:
avg_fn = lambda x: avg_usage(x, architecture)

In [7]:
results = [p.load(open(os.path.join(dir, f), "rb")) for f in files]

In [8]:
"random_removal" in results[0].keys()

True

In [9]:
"random_removal" in results[1].keys()

False

In [10]:
def reduce_result(result):
    reduced = {}
    normal_keys = ["mask angle", "cross inhibit", "random_removal", "accuracy"]

    for k in normal_keys:
        if k in result.keys():
            reduced[k] = float(result[k])
        else:
            reduced[k] = 0.0

    if reduced["mask angle"] > 0.0:
        reduced["reduction"] = "mask angle"
    elif reduced["cross inhibit"] > 0.0:
        reduced["reduction"] = "cross inhibit"
    elif reduced["random_removal"] > 0.0:
        reduced["reduction"] = "random"
    else:
        reduced["reduction"] = "none"
    
    avg_keys = ["matrix usage", "firing rates"]
    for k in avg_keys:
        reduced[k] = avg_fn(result[k])

    reduced["accuracy spiking"] = float(np.max(result["accuracy_spiking"]))
    
    return reduced

In [11]:
reduce_result(results[0])

{'mask angle': 0.45,
 'cross inhibit': 0.0,
 'random_removal': 0.0,
 'accuracy': 0.26100000739097595,
 'reduction': 'mask angle',
 'matrix usage': 0.23743420761024606,
 'firing rates': 0.11709190400843882,
 'accuracy spiking': 0.6340000033378601}

In [12]:
reduced_results = list(map(reduce_result, results))

In [13]:
df = pd.DataFrame(reduced_results)

In [58]:
ma_z = df["mask angle"] == 0
ci_z = df["cross inhibit"] == 0
rnd_z = df["random_removal"] == 0

In [61]:
ma_i = ci_z * rnd_z
ci_i = ma_z * rnd_z
rnd_i = ma_z * ci_z
ctrl_i = ci_z * rnd_z * ma_z

In [60]:
ma = df[ma_i].sort_values("accuracy spiking")
ci = df[ci_i].sort_values("accuracy spiking")
rnd = df[rnd_i].sort_values("accuracy spiking")

In [62]:
baseline = df[ctrl_i]

In [63]:
spk_a = baseline["accuracy spiking"].iloc[0]

In [78]:
spk_a

0.8531000018119812

In [77]:
fig3 = go.Figure()

fig3.add_trace(go.Scatter(x= 1 - ma["firing rates"], y=ma["accuracy spiking"] / spk_a, name="Explicit"))
fig3.add_trace(go.Scatter(x= 1 - ci["firing rates"], y=ci["accuracy spiking"] / spk_a, name="Inhibitory"))
fig3.add_trace(go.Scatter(x= 1 - rnd["firing rates"], y=rnd["accuracy spiking"] / spk_a, name="Random"))

fig3.update_layout(title="Relative Accuracy by Sparsity",
                  xaxis_title="Sparsity",
                  yaxis_title="Relative Accuracy, Spiking",
                  yaxis_range = [0,1.1],
                  xaxis_range = [-0.05, 1.05],
                  width = 900,
                  height = 600,
                  font = dict({"size": 16,}))

In [24]:
px.scatter(ma, x = "mask angle", y = "accuracy spiking")

In [14]:
px.scatter(df, x="firing rates", y="accuracy spiking", color="reduction")